# Inicio

In [2]:
!pip install findspark
!pip install pyspark
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=801a4f430ad75d5ced9a36acbfd6873ddd80c558b9d94d136cd0d27618df7542
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=008155d4564c099c8d1844e4c163999f66b4abffe2b92f144823141ee70758df
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [93]:
import pyspark
import findspark
import spark

findspark.init()

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [97]:
movies = spark.read.option('delimiter', '|').option('header','true').option('inferSchema', 'true').csv('movies.csv')
rating = spark.read.option('delimiter', '|').option('header','true').option('inferSchema', 'true').csv('movie_ratings.csv')

In [104]:
# Calcule la cantidad de películas en las que participó cada actor.
# La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.

movies.groupBy('actor').agg(count('pelicula').alias('conteo')).orderBy(desc('conteo')).show()

+-------------------+------+
|              actor|conteo|
+-------------------+------+
|   Tatasciore, Fred|    38|
|      Welker, Frank|    38|
| Jackson, Samuel L.|    32|
|      Harnell, Jess|    31|
|        Damon, Matt|    27|
|      Willis, Bruce|    27|
|  Cummings, Jim (I)|    26|
|         Hanks, Tom|    25|
|   Lynn, Sherry (I)|    25|
|    McGowan, Mickie|    25|
|    Bergen, Bob (I)|    25|
|      Proctor, Phil|    24|
|   Wilson, Owen (I)|    23|
|        Cruise, Tom|    23|
|         Pitt, Brad|    23|
|Freeman, Morgan (I)|    22|
|Williams, Robin (I)|    22|
|       Depp, Johnny|    22|
|     Morrison, Rana|    22|
|      Diaz, Cameron|    21|
+-------------------+------+
only showing top 20 rows



In [138]:
# Calcule la cantidad de películas producidas cada año.
# La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo.
# La salida debe ordenarse por el conteo en orden descendente.

año_movies = movies.groupBy('año').agg(count_distinct('pelicula').alias('conteo'))

In [141]:
año_movies.select(
    'año',
    when((col('año') > 1900) & (col('año') <= 2000), 'Siglo XX').\
    when((col('año') > 2000) & (col('año') <= 2100), 'Siglo XXI').alias('siglo'),
    'conteo'
).orderBy(desc('conteo')).show()

+----+---------+------+
| año|    siglo|conteo|
+----+---------+------+
|2006|Siglo XXI|    86|
|2004|Siglo XXI|    86|
|2011|Siglo XXI|    86|
|2005|Siglo XXI|    85|
|2008|Siglo XXI|    82|
|2002|Siglo XXI|    81|
|2010|Siglo XXI|    78|
|2000| Siglo XX|    77|
|2003|Siglo XXI|    76|
|2007|Siglo XXI|    75|
|2001|Siglo XXI|    71|
|2009|Siglo XXI|    68|
|1999| Siglo XX|    67|
|1997| Siglo XX|    66|
|1998| Siglo XX|    59|
|1996| Siglo XX|    42|
|2012|Siglo XXI|    32|
|1995| Siglo XX|    25|
|1994| Siglo XX|    16|
|1986| Siglo XX|    16|
+----+---------+------+
only showing top 20 rows



In [143]:
# Obtenga la película con la calificación más alta por año.
# La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.

calif_window = Window.partitionBy('año').orderBy(desc('valoracion'))

rating.withColumn('row', row_number().over(calif_window)).\
select(
    'año',
    'pelicula',
    'valoracion'
).\
filter(
    col('row') == 1
).show()

+----+--------------------+----------+
| año|            pelicula|valoracion|
+----+--------------------+----------+
|1937|Snow White and th...|    2.2207|
|1939|    The Wizard of Oz|    7.9215|
|1940|           Pinocchio|    7.8557|
|1942|               Bambi|    1.5053|
|1946|   Song of the South|     7.602|
|1950|          Cinderella|    9.4226|
|1953|           Peter Pan|    5.4756|
|1954|         Rear Window|   10.7625|
|1955|  Lady and the Tramp|    5.1258|
|1956|Around the World ...|   14.0607|
|1959|     Sleeping Beauty|    6.3919|
|1960|              Psycho|   10.6375|
|1961|One Hundred and O...|    0.6726|
|1962|     The Longest Day|   12.8866|
|1963|It's a Mad Mad Ma...|     6.626|
|1964|        My Fair Lady|     7.587|
|1965|      Doctor Zhivago|    4.9304|
|1966|Who's Afraid of V...|   11.1111|
|1967|     The Dirty Dozen|    13.388|
|1968|        The Love Bug|   13.4383|
+----+--------------------+----------+
only showing top 20 rows

